### Выполнила: Анастасия Бутакова

# Домашнее задание 4.

## Часть 1. Парсинг телеграм-каналов

У вас есть 2 файла на выбор в [этой папке](https://disk.yandex.ru/d/druM43Rd5bdiQg):
- Канал Системный Блокъ
- Канал Нерусский мир

В них сохранена история каналов за 2024 год в html-формате

Подготовьте таблицу, в которой будет:
- отправитель (у нас это будет канал - но в личных чатах туда попадает никнейм отправителя, как в ДЗ_3)
- текст сообщения (его можно немного посчистить (***по желанию***) - например, удалить из него тэги)
- дата
- (***по желанию***) реакции

**Важно!** В Нерусском мире есть небольшие ошибки - посты без автора (можно отловить их и подумать, что с ними делать, или использовать try / except, чтобы их просто проигнорировать)

Примерный результат выполнения всех заданий вы можете найти в той же [папке](https://disk.yandex.ru/d/druM43Rd5bdiQg)

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import codecs
import re

In [ ]:
f = codecs.open("Системный Блокъ.html", 'r', 'utf-8') # читаем код html из файла
telegram = f.read() # записываем его в переменную
f.close()

soup = BeautifulSoup(telegram)
soup

<!DOCTYPE html>
<html>
<head>
<meta charset="utf-8"/>
<title>Exported Data</title>
<meta content="width=device-width, initial-scale=1.0" name="viewport"/>
<link href="css/style.css" rel="stylesheet"/>
<script charset="UTF-8" src="https://gc.kis.v2.scr.kaspersky-labs.com/FD126C42-EBFA-4E12-B309-BB3FDD723AC1/main.js?attr=00VrQMEneJvs38Jm70Rov3kurjhMNcsWm5OyIy_414H-Zr19api0x2_raoBooS-kQu85ggZXWDnYpJ2lYfB2IEbRLmhzcFUjIj2hJcJd-FDfPWtHk70iUoBWXefBk3aPAbiC6AoNsemb5y4ygTVGQLMoe9BiG2oHykPgTy2PnOk2Gp0pDGCxaYgOM6bxXGeRN4Lm6BSSeD6wto68VFdehkxeb_4HCFDTSMEX-OjPLgrOFybJVLW-nXXKCN6IKHz21BMdaVp9cNuf510ZUu40JvRer5WJpforL029DtEqSJvc2YavDk-2PIliEuQlPDgZwuCNY18qGvSoVoyUQ-gnfSh7s7E2n6B_Pp8y2lAc0oE9Xaam8_R0srxxmwxGXywnwmtti-scJY7Ubn9u43e02wWE7_iYnK3JU6c67Uyw7HJ_0dCHF31Pf9OhF6MRpe0ifqim4WBAZFg4o_ThByWJUf2Vo0Sc9rcMhsbX--iUkWOzeiPQQujWST5qWvk36G5MEpveAnK41UmymnsnmfaBqQb9mo9CUmlkTK7cCI_3JhvRmlCCvt2T1hvmhLo1dGGPiZ91IgeNGsi7umOIURd9BRhqcvnE4FTxdZGSoxBs4R3ZKb0HjEmrh2vT__g6RxIkAtNO-OYfUeHftEsVncuvnKFQJ-d4OaCn4e8G0sx

Советы: можно идти по тегам в супе, а можно использовать регулярные выражения к содержимому тега, оставлю одну из возможных подсказок (разбор тегов супом **предпочтительнее**, но задачу можно также решить любым другим способом)

In [ ]:
for i in soup.find_all('div', {'class' : "body"}):
    who = re.findall(r'_здесь_ваша_регулярка_', str(i))
    print(who)

#### Решение

Посмотрим, в каких частях кода находятся искомые элементы:

In [ ]:
# отправитель: <div class="from_name">
# текст сообщения: <div class="text">
# дата: <div class="pull_right date details" title="03.01.2024 19:58:07 "> 19:58 </div>
# реакции: 
#   <div class="reactions">
#   <div class="reaction">
#   <div class="emoji">
#   ❤
         #  </div>
#   <div class="count">
#   31
         #  </div>
#   </div>
# и так далее, реакций может быть много


Задумалась, есть ли смысл каждый раз вытаскивать отправителя, если мы работаем с историей сообщения канала, где он один и тот же. Но помимо Системного Блока сюда закралась Kali Novskaya - если поискать по html-файлу (не знаю, как вывести тут), то видно, что это репост из ее канала. 

In [ ]:
print(set(soup.find_all('div', {"class":"from_name"})))

{<div class="from_name">
Системный Блокъ 
       </div>, <div class="from_name">
Kali Novskaya <span class="date details" title="28.10.2024 16:17:56 "> 28.10.2024 16:17:56</span>
</div>}


В истории сообщений также присутствуют посты, которые представляют собой **опросы или же фото без текста**. В таких случаях у сообщений будут реакции и время, но не будет текста. Учитывая, что мы занимаемся НЛП, думаю, что мы можем проигнорировать существование опросов и фото-постов. Посты в текстом почти удачно сосредоточены в классе "**message default clearfix**" - там есть какой-то один пост без нужной секции, поэтому все равно придется в коде прописать это

#### Код

Сначала - функция для извлечения эмодзи, ибо она довольно объемная, а загромождать код не хочется

In [ ]:
def extract_reactions(message):
    ''' Извлечение информации о реакциях из поста в телеграм. 
        Выдает:
            1. список уникальных реакций
            2. суммарное количество реакций на посте
            3. информацию о том, сколько реакций каждого вида поставлено '''
    
    reactions_section = message.find('div', {'class':'reactions'}) # находим часть сообщения, где хранятся реакции
    reactions = [] # уникальные реакции на посте
    total_count = 0 # сколько всего реакций всех видов - может быть полезным показателем
    reactions_overview = [] # сколько реакций каждого вида на посте (огонек - 3, банан - 1 и т.д.)
    if reactions_section:
        # итерация по каждой отдельной реакции
        for i in reactions_section.find_all('div', {'class':'reaction'}):
            emoji = i.find('div', {'class':'emoji'}).text
            # эмодзи нужно чистить, чтобы удалить знаки разрыва строки и пробелы
            emoji_clean = re.search(r'[^\s]', emoji).group()
            count = int(i.find('div', {'class':'count'}).text) # сколько таких эмодзи на посте
            
            total_count += count
            reactions.append(emoji_clean)
            reactions_overview.append((emoji_clean, count)) # закидываем кортеж из иконки реакции и ее количества на посты
    else:
        pass
    # преобразование инфо о реакциях в более читаемый вид
    reactions_overview_formatted = ', '.join([f'{reaction}: {count}' for reaction, count in reactions_overview])

    return ', '.join(reactions), total_count, reactions_overview_formatted


In [321]:
# пример
variety, total, overview = extract_reactions(messages[1])
print (f'На посте всего {total} реакций. Среди них: {variety}.\nПодробная информация: {overview}')

На посте всего 34 реакций. Среди них: 👍, 🔥, ❤, 🍌, 👾.
Подробная информация: 👍: 14, 🔥: 11, ❤: 7, 🍌: 1, 👾: 1


Итоговый код:

In [ ]:
messages = soup.find_all('div', {'class':'message default clearfix'})
data = []
# поскольку у нас почти всегда один и тот же отправитель, найдем название канала один раз
channel_name = soup.find('div', {"class":"from_name"}).text.strip()

# если есть репост, для того, чтобы вытащить автора оригинала, нужен дополнительный шаг, ведь он находится  внутри 'forwarded message', который лежит внутри 'body'
# а первый подобный find вытащит Системный Блокъ как автора репоста. Учтем в коде

for i in messages:
    name = channel_name
    date = i.find('div', {'class':'pull_right date details'}).get('title')[:10] # здесь есть и другая информация, поэтому делаем срез
    message_text = i.find('div', {'class':'text'})
    # учитываем случаи, когда пост не содержит текста
    if message_text:
        message_text = message_text.text.strip()
    else:
        message_text = 'Ошибка поиска текста в посте'

    react_variety, react_total, react_details = extract_reactions(i)

    # проверка того, что репост или нет
    repost = i.find('div', {'class':'forwarded body'}) 
    if repost:
        name = repost.find('div', {"class":"from_name"}) # меняем отправителя на автора оригинала


    data.append((name, date, message_text, react_variety, react_total, react_details))


In [322]:
# датафрейм
df = pd.DataFrame(data, columns=['Отправитель', 'Дата', 'Текст поста', 'Реакции', 'N реакций', 'Подробная информация о реакциях'])
df.head()

,Отправитель,Дата,Текст поста,Реакции,N реакций,Подробная информация о реакциях
0,Системный Блокъ,03.01.2024,Что происходит в интернете: Оксана Мороз о циф...,"❤, 🎄, 👍, 🔥, 👏",42,"❤: 31, 🎄: 5, 👍: 3, 🔥: 2, 👏: 1"
1,Системный Блокъ,06.01.2024,"Танцы, эрос и зачатие: о чем писали «Платоновс...","👍, 🔥, ❤, 🍌, 👾",34,"👍: 14, 🔥: 11, ❤: 7, 🍌: 1, 👾: 1"
2,Системный Блокъ,10.01.2024,Читаем секретные письма опальной королевы: ист...,"🔥, ❤, 👍, 👾, 👀",39,"🔥: 15, ❤: 12, 👍: 8, 👾: 3, 👀: 1"
3,Системный Блокъ,11.01.2024,От Эдисона до Spotify: история форматов музыки...,"🔥, 👍, ❤, 👻",46,"🔥: 20, 👍: 16, ❤: 9, 👻: 1"
4,Системный Блокъ,15.01.2024,"Больше, чем энциклопедия: Википедии 23 года!По...","🍾, ❤, 👍, ⚡, 👏, 👎",46,"🍾: 17, ❤: 11, 👍: 10, ⚡: 5, 👏: 2, 👎: 1"


In [319]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 204 entries, 0 to 203
Data columns (total 6 columns):
 #   Column                           Non-Null Count  Dtype 
---  ------                           --------------  ----- 
 0   Отправитель                      204 non-null    object
 1   Дата                             204 non-null    object
 2   Текст поста                      204 non-null    object
 3   Реакции                          204 non-null    object
 4   N реакций                        204 non-null    int64 
 5   Подробная информация о реакциях  204 non-null    object
dtypes: int64(1), object(5)
memory usage: 9.7+ KB


In [323]:
df.to_excel('sysblock.xlsx')

## Часть 2

Парсинг сайта, на выбор:
- [Детские вопросы](https://elementy.ru/email) в журнале "Элементы" (немного легче)
- [Рубрика "Ускользающий мир"](https://www.nkj.ru/special/mir/) журнала "Наука и жизнь"

Общий алгоритм:
- смотрим на 1 и последнюю страницу и с помощью range(от, до+1) генерируем ссылки на страницы
- проходим по каждой странице и парсим ссылки на новости (важно собрать нужные и очистить от лишних!). На момент начала задания было 88 Детских вопроса и 213 текстов Ускользающего мира (ближе к дедлайну их может стать немного больше)
- переходим к каждой новости и парсим из нее:
  - заголовок, автор ответа, текст, ссылку для Детских вопросов
  - заголовок, дату, текст и ссылку для Ускользающего мира (***опционально*** - также автора, источник, тэги, но учтите, что они есть не в каждой статье)
- сохраняем результаты в датафрейм, не забудьте озаглавить столбцы

Поскольку мы парсим реальные сайты, вежливый парсинг с time и задержкой в несколько секунд будет плюсом!

In [23]:
import time

In [ ]:
# ваш код

# генерация ссылок на страницы сайта
base_page = 'https://www.nkj.ru/special/mir/index.php?PAGEN_2=6&mobile=N&PAGEN_1='
links = []
for i in range(1,12):
    links.append(base_page + str(i))
print(links)


['https://www.nkj.ru/special/mir/index.php?PAGEN_2=6&mobile=N&PAGEN_1=1', 'https://www.nkj.ru/special/mir/index.php?PAGEN_2=6&mobile=N&PAGEN_1=2', 'https://www.nkj.ru/special/mir/index.php?PAGEN_2=6&mobile=N&PAGEN_1=3', 'https://www.nkj.ru/special/mir/index.php?PAGEN_2=6&mobile=N&PAGEN_1=4', 'https://www.nkj.ru/special/mir/index.php?PAGEN_2=6&mobile=N&PAGEN_1=5', 'https://www.nkj.ru/special/mir/index.php?PAGEN_2=6&mobile=N&PAGEN_1=6', 'https://www.nkj.ru/special/mir/index.php?PAGEN_2=6&mobile=N&PAGEN_1=7', 'https://www.nkj.ru/special/mir/index.php?PAGEN_2=6&mobile=N&PAGEN_1=8', 'https://www.nkj.ru/special/mir/index.php?PAGEN_2=6&mobile=N&PAGEN_1=9', 'https://www.nkj.ru/special/mir/index.php?PAGEN_2=6&mobile=N&PAGEN_1=10', 'https://www.nkj.ru/special/mir/index.php?PAGEN_2=6&mobile=N&PAGEN_1=11']


In [ ]:
# выгрузка ссылок на новости со страниц
base_news = 'https://www.nkj.ru'
links_news = []
for i in links:
    page = requests.get(i)
    soup = BeautifulSoup(page.text)
    links_page = [base_news + link.get('href') for link in soup.find_all('a') if link.parent.name == 'h2']
    links_news = links_news + links_page
    time.sleep(2) # пощадим сайт


In [26]:
len(links_news)

220

In [ ]:
# заголовок <h1 itemprop="headline">
# дата <time class="nomer-god news-time" datetime="2025-04-25T16:14+03:00" style="display: inline; font-style: normal; color: #999">25 апреля 2025</time>
# текст body > div.container.inner > div > div.span9 > article > main > p:nth-child(5)
# ссылка  
# автор - <p class='author'>
# источник <p class='istok'>
# тэги - <span class="article-tag" style="padding-right: 1em"><a href="/search/tag.php?tags=%D0%A3%D1%81%D0%BA%D0%BE%D0%BB%D1%8C%D0%B7%D0%B0%D1%8E%D1%89%D0%B8%D0%B9+%D0%BC%D0%B8%D1%80">#Ускользающий мир</a></span>

In [101]:
from tqdm import tqdm

In [213]:
data = []

for i in tqdm(links_news, desc = 'Прогресс парсинга'):
    page = requests.get(i)
    soup = BeautifulSoup(page.text)

    title = soup.find('h1').text.strip()
    date = soup.find('time').text
    article_body = soup.find('main')
    # ищем элементы с тегом p, у которых нет атрибутов внутри - чтобы не захватывать краткое содержание
    paragraphs = [el.text.strip() for el in article_body.find_all('p') if not el.attrs]
    article_text = ' '.join(paragraphs)
    
    # в одной статье текст находится не внутри тегов <p> в <main>, а просто в самом <main>
    # Введем проверку на наличие любого символа кроме пробельных в вытащенном тексте + альтернативный путь для поиска текста
    if not re.search(r'[^\s]', article_text):
        article_text = re.sub(r'\s+', ' ', article_body.text.strip())
        
    # элементы, которые есть не у всех
    nan = 'нет данных'
    author = soup.find('p', {'class' : 'author'})
    if author:
        author = author.text.strip()
        author = re.sub(r'Автор:\xa0', '', author)
    else:
        author = nan
    source = soup.find('p', {'class' : 'istok'})
    if source:
        source = source.text.strip()
        source = re.sub(r'Источник:\xa0', '', source)
    else:
        source = nan
    tags = ' '.join([el.text.strip() for el in soup.find_all('span', {'class' : 'article-tag'})]) if soup.find_all('span', {'class' : 'article-tag'}) else nan

    data.append((title, date, article_text, i, source, author, tags))
    time.sleep(2)




Прогресс парсинга: 100%|██████████| 220/220 [09:36<00:00,  2.62s/it]


In [214]:
df2 = pd.DataFrame(data, columns = ['Заголовок', 'Дата', 'Текст', 'Ссылка', 'Источник', 'Автор', 'Теги'])
df2.head()

,Заголовок,Дата,Текст,Ссылка,Источник,Автор,Теги
0,Мониторинг таймырской популяции северного оленя,25 апреля 2025,Последние четверть века на полуострове Таймыр...,https://www.nkj.ru/special/mir/54342/,нет данных,нет данных,#Ускользающий мир #мониторинг #северный олень ...
1,Безопасные миграции,23 апреля 2025,В России принимаются дополнительные меры по ох...,https://www.nkj.ru/special/mir/54329/,нет данных,нет данных,#Ускользающий мир #заповедники #миграции живот...
2,Сто крапчатых сусликов выпущено в охранной зон...,21 апреля 2025,До середины XX века крапчатый суслик был мно...,https://www.nkj.ru/special/mir/54320/,нет данных,нет данных,#Ускользающий мир #заповедники #суслик #животн...
3,В заповеднике «Чёрные земли» обнаружили кости ...,18 апреля 2025,Согласно заключению экспертов-палеонтологов и...,https://www.nkj.ru/special/mir/54308/,нет данных,нет данных,#Ускользающий мир #палеонтология #степь #мегаф...
4,Влияние климатических изменений на численность...,16 апреля 2025,Ежегодные наблюдения за краснокнижным эндеми...,https://www.nkj.ru/special/mir/54297/,нет данных,нет данных,#Ускользающий мир #заповедники #учёты #сурки


In [215]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 220 entries, 0 to 219
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Заголовок  220 non-null    object
 1   Дата       220 non-null    object
 2   Текст      220 non-null    object
 3   Ссылка     220 non-null    object
 4   Источник   220 non-null    object
 5   Автор      220 non-null    object
 6   Теги       220 non-null    object
dtypes: object(7)
memory usage: 12.2+ KB


In [216]:
df2.to_excel('uskolz_mir.xlsx')